In [1]:
import numpy as np
from scipy.optimize import brentq, fixed_point
import pint

## Raindrop shape

In order to calculate terminal velocity we need to find $\frac{a}{b}$, which is uniquely determined from $r_{eq}$ with following equation:

$$

r_{eq} = \sqrt{\frac{\sigma_{c-air}}{g(\rho_{c,l} - \rho_{air})}}\left(\frac{b}{a}\right)^{-\frac{1}{6}}\sqrt{\left(\frac{b}{a}\right)^{-2} - 2\left(\frac{b}{a}\right)^{-\frac{1}{3}} + 1}

$$

This can be done by transforming it into form $F(x) = 0$ where the variable $x = \frac{b}{a}$ and finding roots of $F$.

$$

F\left(\frac{b}{a}\right) = \sqrt{\frac{\sigma_{c-air}}{g(\rho_{c,l} - \rho_{air})}}\left(\frac{b}{a}\right)^{-\frac{1}{6}}\sqrt{\left(\frac{b}{a}\right)^{-2} - 2\left(\frac{b}{a}\right)^{-\frac{1}{3}} + 1} - r_{eq} = 0


$$

In [2]:
ureg = pint.UnitRegistry()

SIGMA_WATER_AIR = 0.073 #* ureg.newton / ureg.meter
GRAVITY = 9.81 #* ureg.meter / ureg.second**2
RHO_WATER = 1000.0 #* ureg.kilogram / ureg.meter**3
RHO_AIR = 1.205 #* ureg.kilogram / ureg.meter**3

def calculate_shape_ratio(r_eq):
    def F(x):
        return np.sqrt(SIGMA_WATER_AIR / (GRAVITY * (RHO_WATER - RHO_AIR))) * x**(-1/6) * \
               np.sqrt(x**(-2) - 2 * x**(-1/3) + 1) - r_eq
    return brentq(F, 1e-9, 1.0 - 1e-9)

r_eq = 0.004 #* ureg.meter

shape_ratio = calculate_shape_ratio(r_eq)
shape_ratio

0.5592937418316654

$$
f_{SA} = \begin{cases} 0.5 \left(\frac{b}{a}\right)^{-2/3} + \left(\frac{b}{a}\right)^{4/3} (4\epsilon)^{-1}ln\left[\frac{1 + \epsilon}{1 - \epsilon}\right], & b/a < 1 \\ 1, & b/a = 1 \end{cases}
$$,
where $\epsilon = \sqrt{1 - (b / a)^2}$

In [ ]:
def calculate_fSA(shape_ratio):
    epsilon = np.sqrt(1 - shape_ratio**2)
    if shape_ratio < 1:
        return 0.5 * (shape_ratio ** (-2/3)) + shape_ratio ** (4/3) * \
            np.log((1 + epsilon) / (1 - epsilon)) / (4 * epsilon)
    elif shape_ratio == 1:
        return 1.0
    else:
        raise ValueError("Shape ratio must be in (0, 1]")

fSA = calculate_fSA(shape_ratio)
fSA

3.0917676852427265

$$

C_{shape}=1+1.5(f_{SA}-1)^{0.5}+6.7(f_{SA}-1)

$$

In [4]:
def calculate_C_shape(fSA):
    return 1 + 1.5 * (fSA - 1)**0.5 + 6.7 * (fSA - 1)

C_shape = calculate_C_shape(fSA)
C_shape

17.18428519160963

$$
C_{D}=\left(\frac{24}{Re}\left(1+0.15Re^{0.687}\right)+0.42\left(1+4.25\times10^{4}Re^{-1.16}\right)^{-1}\right)C_{shape}
$$

In [5]:
AIR_VISCOSITY = 1.81e-5

def calculate_CD(C_shape, vT, r_eq):
    Re = vT * 2 * r_eq * RHO_AIR / AIR_VISCOSITY
    return ((24 / Re) * (1 + 0.15 * Re**0.687) + 0.42 * (1 + 4.25 * 10**4 * Re**-1.16)**-1) * C_shape

CD = calculate_CD(C_shape, 6.5, r_eq)
CD


6.611423847310984

$$
v_t = -\sqrt{\frac{8}{3}\frac{(\rho_{c,l} - \rho_{air})}{\rho_{air}}\frac{g}{C_D(v)}\left(\frac{b}{a}\right)^{\frac{2}{3}}r_{eq}}
$$

In [6]:
def calculate_terminal_velocity(Cd, shape_ratio, r_eq):
        return np.sqrt(8/3 * ((RHO_WATER - RHO_AIR) / RHO_AIR) * (GRAVITY / Cd) * shape_ratio**(2/3) * r_eq)

def find_terminal_velocity(r_eq, C_shape, shape_ratio):
    v0 = 0.001
    def f(vT):
        vT = np.abs(vT)
        Cd = calculate_CD(C_shape, vT, r_eq)
        return calculate_terminal_velocity(Cd, shape_ratio, r_eq)

    return fixed_point(f, v0)

v_terminal = find_terminal_velocity(r_eq, C_shape, shape_ratio)
v_terminal

array(2.84635771)

In [7]:
CD = calculate_CD(C_shape, v_terminal, r_eq)
np.sqrt(8/3 * ((RHO_WATER - RHO_AIR) / RHO_AIR) * (GRAVITY / CD) * shape_ratio**(2/3) * r_eq)

2.8463577145351304

In [8]:
print(r_eq)
print(C_shape)
print(shape_ratio)
print(v_terminal)
print(fSA)

0.004
17.18428519160963
0.5592937418316654
2.8463577145351304
3.0917676852427265
